In [1]:
import yaml
import requests

In [4]:
API_KEY = yaml.full_load(open('creds.yaml', 'r'))['maps_api']

In [7]:
req = requests.get("https://maps.googleapis.com/maps/api/directions/json?avoid=highways&destination=Montreal&mode=bicycling&origin=Toronto&key={}".format(API_KEY))

In [10]:
req.json()

{'geocoded_waypoints': [{'geocoder_status': 'OK',
   'place_id': 'ChIJpTvG15DL1IkRd8S0KlBVNTI',
   'types': ['locality', 'political']},
  {'geocoder_status': 'OK',
   'place_id': 'ChIJDbdkHFQayUwR7-8fITgxTmU',
   'types': ['locality', 'political']}],
 'routes': [{'bounds': {'northeast': {'lat': 45.4969544, 'lng': -73.5630455},
    'southwest': {'lat': 43.6532565, 'lng': -79.3858556}},
   'copyrights': 'Map data ©2022 Google',
   'legs': [{'distance': {'text': '600 km', 'value': 599960},
     'duration': {'text': '1 day 6 hours', 'value': 109452},
     'end_address': 'Montreal, QC, Canada',
     'end_location': {'lat': 45.4969544, 'lng': -73.5666869},
     'start_address': 'Toronto, ON, Canada',
     'start_location': {'lat': 43.6532565, 'lng': -79.38303979999999},
     'steps': [{'distance': {'text': '0.1 km', 'value': 99},
       'duration': {'text': '1 min', 'value': 61},
       'end_location': {'lat': 43.6524799, 'lng': -79.38296489999999},
       'html_instructions': 'Head <b>south